In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz as gp
from pygal.maps.world import World
from IPython.display import SVG, display
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans

# A customer segmentation exercise

The data we use is originally from [Kaggle](https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python). We start off by importing the data.

In [ ]:
customers=pd.read_csv("mall_customers.csv")
customers

We remove the customer ID, since this doesn't hold any relevant information:

In [ ]:
customers=customers.drop(columns=["CustomerID"])

We can only work with numerical variables, so let's one-hot-encode the gender:

In [ ]:
customers = pd.get_dummies(customers,drop_first=True,columns=['Gender'])

Finally, we note the very different scales. Scaling would be one way, but given that we also have the binary gender variable, it may be more sensible to only normalize:

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
customers = pd.DataFrame(min_max_scaler.fit_transform(customers.values), columns=customers.columns, index=customers.index)
customers

## Using hierarchical clustering on the customers

1. Using `scipy`, create a dendrogram `Z` with `linkage(customers,method="average")`, then visualize it with `dendogram(Z)`. How many clusters do you think we should use?

2. Repeat the process using `method="ward"`. How many clusters do you think we should use in this case?

3. We take 4 clusters. Using the code below, we obtain the labels for the data and add them to the `customers` dataframe to obtain `customers_hier`. We then plot the average value of the scores taken on for each feature in that cluster and put this into a heatmap to better visualize. What kind of clusters appear?

In [ ]:
labels=fcluster(Z, 4, criterion='maxclust') 

In [ ]:
customers_hier=customers.copy()
customers_hier["Labels"]=labels

customers_hier=customers_hier.groupby(customers_hier["Labels"]).mean().reset_index()
customers_hier=customers_hier.drop(columns=["Labels"]).set_index(np.arange(1,5))
sns.heatmap(customers_hier,cmap="PiYG")
plt.show()

4. Instead of just plotting the average values, we can draw a snake plot that also includes the range of values around the mean. For this, we need to "melt" the dataframe, that is, we need to have all attribute values in a single column, with an additional column specifying the attribute to which the value belongs

In [ ]:
customers_hier=customers.copy()
customers_hier["Labels"]=labels
customers_hier_melt = pd.melt(customers_hier.reset_index(),
                            id_vars=['Labels'],
                            value_vars=['Age','Annual Income (k$)','Spending Score (1-100)','Gender_Male'],
                            var_name='Attribute',value_name='Value')
plt.title('Snake plot of the clusters')
sns.lineplot(x="Attribute", y="Value", hue='Labels', data=customers_hier_melt)
plt.show()

## Using K-means clustering on the customers

5. Use `KMeans()` instead with `n_clusters=4`. Call this model `kmeans` and fit it to the `customers` data.
From our `kmeans` model, we get the labels with `kmeans.labels_`. Use this to create a similar heatmap and snakeplot. Do the clusters obtained show similarities?

6. In K-means clustering, we usually look at inertia to see how many clusters to choose. The inertia measures the sum of distances between each point and the centroid of its cluster. Run the code below to find out how inertia changes with the number of clusters. How many clusters would you pick? Why do you think it's not a good idea to go for the number of clusters which gives the smallest inertia?

In [ ]:
inertia_K=[]
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(customers)
    inertia_K.append(kmeanModel.inertia_)
plt.plot(K,inertia_K)
plt.show()

## Developing a targeted marketing campaign

7. Run again the code with K-means clustering and four clusters. Based on this, together with an overview of the size of the different clusters (below), what marketing strategy would you suggest to the mall provider?

In [ ]:
sns.histplot(customers_km['Labels'])
plt.show()